# Raster data: Watershed Delineation

Documentation reference: https://support.esri.com/en-us/knowledge-base/how-to-create-a-watershed-model-using-the-hydrology-too-000012346

Data Required:
1. Digital Elevation Model
2. Polygon of study area
3. Pour Points shapefile

What is watershed delineation?
It is the process through which we can identify which area contributes to the outflow of water through a specific point (know as pour points).

Steps/Tools used:
1. Clip: The DEM raster needs to be clipped as per the study area polygon.
2. Fill: Smoothens out the sudden sinks and peaks (outliers) in the raster, so we get a continuous surface.
3. Flow direction: Determines the direction of flow from each cell to its steepest downslope neighbor.
4. Flow accumulation: Calculates the accumulated flow to each cell, as determined by the accumulated weight of all cells that flow into each downslope cell.
5. Snap to pour points: 

In [23]:
# Setting up environment and input data

import arcpy


# Set up the workspace with overwrite output set to true
working_dir = r"C:\Users\nomitrawat\Documents\Training\Data\Raster\Dehradun"
arcpy.env.workspace = working_dir
arcpy.env.overwriteOutput = True

# Input datasets
dem = "DEM.tif"
polygon = "tehri_polygon.shp"
pour_point = "pour_point.shp"

# Check the dataset
if arcpy.Exists(dem):
    print("Found DEM")

if arcpy.Exists(polygon):
    print("Found polygon")

if arcpy.Exists(pour_point):
    print("Found pour points")

Found DEM
Found polygon
Found pour points


# Watershed delineation

In [24]:
#Clip

clipped = arcpy.management.Clip(
    in_raster = dem,
    in_template_dataset = polygon,
    out_raster = "clipped_dem",
    clipping_geometry="ClippingGeometry"
)

clipped

<Result 'C:\\Users\\nomitrawat\\Documents\\Training\\Data\\Raster\\Dehradun\\clipped_dem'>

In [26]:
#Fill

fill = arcpy.sa.Fill(clipped)
fill.save("filled")
#note
#name of the file provided as the argument
#saves the file in the workspace

In [27]:
#Flow direction

flow_dir = arcpy.sa.FlowDirection(fill)
flow_dir.save("flow_dir")
#note
#name should have less than 13 characters for ESRI grid format

In [28]:
#Flow accumulation

flow_acc = arcpy.sa.FlowAccumulation(flow_dir)
flow_acc.save("flow_acc")

In [30]:
#Snap pour point

snap_pour = arcpy.sa.SnapPourPoint(pour_point, flow_acc, 0)
snap_pour.save("snap_pour")
#note
#snap distance needs to be provided

In [31]:
#Watershed

watershed = arcpy.sa.Watershed(flow_dir, snap_pour)
watershed.save("watershed")

In [32]:
#Raster to polygon

watershed_poly = arcpy.conversion.RasterToPolygon(watershed, "watershed.shp")